## Evaluation of Models on Incident Prediction

In [54]:
import numpy as np
import torch

from torchmetrics import Accuracy
from torchmetrics.functional import precision_recall
from tqdm import tqdm
from datetime import datetime as dt
from matplotlib import pyplot as plt

from train import create_parser
from models import *

### 1. Load Model and Data

In [33]:
parser = create_parser()
args = parser.parse_args(args=[])

args.use_density = True
args.use_truck_spd = True
args.use_pv_spd = True
args.use_speed = True
args.use_expectation = True
args.device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [34]:
traffic_model = TrafficModel(args)
traffic_model_path = "./checkpoints/finetune/best_exp_tmc_T_T_T_7_6_5_T.pt"
base_model = TrafficSeq2Seq(args)
base_model_path = "./checkpoints/base/best_exp_tmc_T_T_T_7_6_5_T.pt"
X_path = "./data/new_X.npy"
Y_path = "./data/new_Y_tmc.npy"
sigm = torch.nn.Sigmoid()

In [35]:
with open(base_model_path, "rb") as f_base_model, open(traffic_model_path, "rb") as f_traffic_model:
    # load state dict
    state_dict_base = torch.load(f_base_model, map_location=args.device)
    state_dict_traffic = torch.load(f_traffic_model, map_location=args.device)

    # populate state dict into models and log
    base_model.load_state_dict(state_dict_base)
    traffic_model.load_state_dict(state_dict_traffic)

In [36]:
days = 164
daily_slots = 180

input = torch.from_numpy(np.load(X_path)).float()  # (29520, 1796)
output = torch.from_numpy(np.load(Y_path)).float()   # (29520, 70, 4)

X = []  # serve as model input
Y = []  # serve as model input
target = []  # ground truth

for d in range(days):
    for i in range(daily_slots-args.in_seq_len+1):
        X.append(input[d*days+i : d*days+i+args.in_seq_len, :])
        Y.append(output[d*days+i : d*days+i+args.in_seq_len, ...])
X = torch.stack(X, dim=0)  # (28536, 7, 1796)
Y = torch.stack(Y, dim=0)  # (28536, 7, 70, 4)

for d in range(days):
    for i in range(1, daily_slots-args.out_seq_len+1):
        target.append(output[d*days+i : d*days+i+args.out_seq_len, ...])
target = torch.stack(target, dim=0)  # (28536, 6, 70, 4)
inc_target = target[...,3].int()  # (28536, 6, 70)
spd_target = target[..., :3]  # (28536, 6, 70, 3)


### 2. Generate Model Inference

In [37]:
batch_size = 24
traffic_spd_pred, traffic_inc_pred, base_spd_pred = [], [], []
with torch.no_grad():
    traffic_model.eval()
    base_model.eval()
    for i in tqdm(range(1189)):
        temp_X = X[(i*24) : (i+1)*24]
        temp_Y = Y[(i*24) : (i+1)*24]
        temp_traffic_spd_pred, temp_traffic_inc_pred, _ = traffic_model(temp_X, temp_Y)
        temp_base_spd_pred, _, _ = base_model(temp_X, temp_Y)

        traffic_spd_pred.append(temp_traffic_spd_pred)
        traffic_inc_pred.append(temp_traffic_inc_pred)
        base_spd_pred.append(temp_base_spd_pred)

traffic_spd_pred = torch.cat(traffic_spd_pred, dim=0)  # (28536, 6, 210)
traffic_inc_pred = sigm(torch.cat(traffic_inc_pred, dim=0))  # (28536, 6, 70)
base_spd_pred = torch.cat(base_spd_pred, dim=0)  # (28536, 6, 210)

100%|██████████| 1189/1189 [00:36<00:00, 32.23it/s]


In [38]:
np.save("traffic_spd_pred_tmc.npy", traffic_spd_pred.cpu().detach().numpy())
np.save("traffic_inc_pred_tmc.npy", traffic_inc_pred.cpu().detach().numpy())
np.save("base_spd_pred_tmc.npy", base_spd_pred.cpu().detach().numpy())

### 3. Effect of Incident Prediction Threshold on Accuracy VS Recall

In [39]:
thresholds = [0.05*i for i in range(20)]
accu = []
recall = []
for t in thresholds:
    accu_metric = Accuracy(t)
    accu.append(accu_metric(sigm(traffic_inc_pred), inc_target))
    recall.append(precision_recall(sigm(traffic_inc_pred), inc_target, threshold=t)[1])

In [40]:
thres_accu_recall = [np.array(thresholds), torch.tensor(accu).numpy(), torch.tensor(recall).numpy()]
thres_accu_recall = np.stack(thres_accu_recall, axis=0)
np.save("thres_accu_recall", thres_accu_recall)

### 4. Case Study of Timeliness
- Here we use the actual event of Waze report: TMC 104-4441 (19th out segment, index 18) 17:15:00 on Mar 5, 2019 (24th date, index 23, starting from Feb 10, 2019)

- We check our model prediction on time slots 16:50:00~17:15:00 on Mar 5, 2019

In [80]:
time_idx = 23*174 + 129
seg_idx = 18
selected_inc_pred = traffic_inc_pred[time_idx, :, seg_idx]
selected_inc_target = inc_target[time_idx, :, seg_idx]

selected_traffic_spd_pred = traffic_spd_pred.reshape(-1, args.out_seq_len, args.out_dim, 3)[time_idx, :, seg_idx, :]
selected_spd_target = spd_target[time_idx, :, seg_idx, :]
selected_base_spd_pred = base_spd_pred.reshape(-1, args.out_seq_len, args.out_dim, 3)[time_idx, :, seg_idx, :]

In [81]:
selected_inc_pred, selected_inc_target

(tensor([0.7054, 0.7160, 0.7686, 0.7942, 0.8103, 0.7997]),
 tensor([0, 0, 0, 0, 0, 0], dtype=torch.int32))

In [85]:
selected_traffic_spd_pred.T

tensor([[67.1597, 66.9008, 64.9611, 65.8758, 65.0900, 64.4586],
        [64.0378, 63.8719, 63.3924, 64.1188, 63.9558, 63.6668],
        [68.5004, 68.0913, 67.2742, 68.2603, 67.9251, 67.3199]])

In [83]:
selected_spd_target.T

tensor([[63.0000, 59.0000, 64.0000, 62.0000, 63.0000, 63.0000],
        [59.9394, 60.0000, 60.0000, 64.0000, 56.0000, 59.9394],
        [63.0000, 59.0000, 65.0000, 61.0000, 65.0000, 63.0000]])

In [84]:
selected_base_spd_pred.T

tensor([[64.6162, 66.2748, 68.2529, 69.2244, 69.3502, 69.1799],
        [63.8547, 64.8876, 66.5837, 67.2956, 67.1768, 66.9250],
        [65.5144, 67.2508, 69.2324, 70.1771, 70.3065, 70.1411]])

In [123]:
# inference on 16:45:00, as well as the following time period (17:20:00 ~ 17:45:00), 
lag = -1 # 6
selected_traffic_spd_pred_2 = traffic_spd_pred.reshape(-1, args.out_seq_len, args.out_dim, 3)[time_idx+lag, :, seg_idx, :]
selected_spd_target_2 = spd_target[time_idx+lag, :, seg_idx, :]
selected_base_spd_pred_2 = base_spd_pred.reshape(-1, args.out_seq_len, args.out_dim, 3)[time_idx+lag, :, seg_idx, :]

In [124]:
traffic_inc_pred[time_idx-1, :, seg_idx]

tensor([0.7280, 0.7357, 0.7709, 0.7378, 0.7393, 0.7421])

In [125]:
selected_traffic_spd_pred_2.T

tensor([[66.8232, 66.8523, 65.1955, 63.1000, 64.8696, 65.2021],
        [64.6081, 64.8071, 64.0083, 62.9499, 63.9544, 64.0970],
        [68.7255, 68.4531, 67.1062, 66.4765, 67.6330, 67.7268]])

In [126]:
selected_spd_target_2.T

tensor([[68.0000, 63.0000, 59.0000, 64.0000, 62.0000, 63.0000],
        [64.6964, 59.9394, 60.0000, 60.0000, 64.0000, 56.0000],
        [68.0000, 63.0000, 59.0000, 65.0000, 61.0000, 65.0000]])

In [127]:
selected_base_spd_pred_2.T

tensor([[63.9186, 65.0571, 67.0426, 68.2490, 68.7815, 68.9281],
        [63.2506, 63.7744, 65.6257, 66.6814, 66.9713, 66.9547],
        [64.8901, 66.0237, 67.9922, 69.1591, 69.6962, 69.8517]])